# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
print('Workspace Name: ' + ws.name,
      'Resource group: ' + ws.resource_group,
      'Azure Region: ' + ws.location,
      'Subscription Id: ' + ws.subscription_id, sep = '\n')

In [ ]:
# choose a name for experiment
experiment_name = 'Framingham-Experiment'

experiment=Experiment(ws, experiment_name)
run = exp.start_logging()

In [ ]:
# Choose a name for your CPU cluster
amlcompute_cluster_name = "Compute-Cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

In [ ]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
found = False
key = "Framingham"
description_text = "Frahmingham Heart Disease Dataset"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/Haris-09/Coronary-Heart-Disease-Prediction-Capstone-Project/master/starter_file/framingham.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

In [ ]:
df = dataset.to_pandas_dataframe()
df.describe()

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {"experiment_timeout_minutes": 30,
                   "max_concurrent_iterations": 4,
                   "primary_metric" : 'accuracy',
                   "verbosity": logging.INFO}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="TenYearCHD",   
                             path = './',
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()

In [ ]:
remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_automl_run, best_automl_model = remote_run.get_output()

print('Best Run Id: 'best_automl_run.id,
     'Best Model Name: 'best_automl_run.properties['model_name'],
     )

In [ ]:
best_run_metrics = best_run.get_metrics()

for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name,":" , metric)

In [ ]:
#TODO: Save the best model
automodel = best_automl_run.register_model(model_name='best_automl_model', 
                                    model_path='./model.pkl',
                                    tags={'Method':'AutoML'},
                                    properties={'Accuracy': best_run_metrics['Accuracy']})

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# Download scoring file 
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

# Download environment file
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'env.yml')

In [ ]:
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               description='Framingham Heart Disease Prediction with AutoML')

In [ ]:
inference_config = InferenceConfig(entry_script="score.py", environment=best_run.get_environment())

service = Model.deploy(workspace=ws, 
                       name='automl-webservice', 
                       models=[automodel], 
                       inference_config=inference_config, 
                       deployment_config=aciconfig)

In [ ]:
service.wait_for_deployment(show_output=True)

In [ ]:
print("Service State: ",service.state)
print("Scoring URI: ",service.scoring_uri)
print("Swagger URI: ",service.swagger_uri)

In [ ]:
!python logs.py

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
data = {
    "data": [
        {
            'gender': 0,
            'age': 55,
            'education': 4 ,
            'currentSmoker': 1,
            'cigsPerDay': 20,
            'BPMeds': 0,
            'prevalentStroke': 0,
            'prevalentHyp': 0,
            'diabetes': 0,
            'totChol': 285,
            'sysBP': 175,
            'diaBP': 84,
            'BMI': 25.50,
            'heartRate': 93,
            'glucose': 87
        },
        {
            'gender': 1,
            'age': 40,
            'education': 4 ,
            'currentSmoker': 0,
            'cigsPerDay': 30,
            'BPMeds': 0,
            'prevalentStroke': 0,
            'prevalentHyp': 0,
            'diabetes': 0,
            'totChol': 228,
            'sysBP': 121,
            'diaBP': 110,
            'BMI': 30.30,
            'heartRate': 80,
            'glucose': 80
        }
    ]
}

data_sample = json.dumps(data)

In [ ]:
scoring_uri = service.scoring_uri
input_data = data_sample

# Set the content type
headers = {'Content-Type': 'application/json'}

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
response.json()

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
service.get_logs()

In [ ]:
service.detete()